In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool,CatBoost, CatBoostRegressor
from collections import Counter

In [2]:
cust_embs=np.load('cust_embs.npy', allow_pickle=True).item()

In [3]:
def metric(predicts, Ys):
    counter = Counter(Ys)
    max_value = counter['view']*0.1+counter['skip']*0.1+counter['like']*0.5+counter['dislike']*10
    value = 0
    for y,p in zip(Ys,predicts):
        if y == "dislike":
            value-=10*p
        if y == "like":
            value+=0.5*p
        if y =='view':
            value+=0.1*p
        if y =='skip':
            value-=0.1*p
    return value/max_value
            

In [4]:
data_path = 'dataset/'
customer_train = pd.read_csv(f'{data_path}customer_train.csv')
customer_test = pd.read_csv(f'{data_path}customer_test.csv')
stories_description = pd.read_csv(f'{data_path}stories_description.csv')
stories_reaction_test = pd.read_csv(f'{data_path}stories_reaction_test.csv')
stories_reaction_test['event_dttm']=pd.to_datetime(stories_reaction_test['event_dttm'],infer_datetime_format=True)
stories_reaction_train = pd.read_csv(f'{data_path}stories_reaction_train.csv')
stories_reaction_train['event_dttm']=pd.to_datetime(stories_reaction_train['event_dttm'],infer_datetime_format=True)
transactions = pd.read_csv(f'{data_path}transactions.csv')


sample_submit = pd.read_csv(f'{data_path}sample_submit.csv')

In [5]:
train_and_val = stories_reaction_train.merge(customer_train, on='customer_id', how='left')
test = stories_reaction_test.merge(customer_test, on='customer_id', how='left')

train_and_val['gender_cd']=train_and_val.gender_cd.map(lambda x: 1 if x =="M" else(0 if x =="F" else -1))
test['gender_cd']=test.gender_cd.map(lambda x: 1 if x =="M" else(0 if x =="F" else -1))
test['children_cnt']=test.children_cnt
train_and_val['children_cnt']=train_and_val.children_cnt

In [6]:
train_dates = stories_reaction_train['event_dttm'].astype(np.int64)
test_dates = stories_reaction_test['event_dttm'].astype(np.int64)
period = train_dates.max() - (test_dates.max() - test_dates.min())

In [7]:
val = train_and_val[train_and_val.event_dttm.astype(np.int64) > period]
train = train_and_val[train_and_val.event_dttm.astype(np.int64) <= period]

In [8]:
train=train.sort_values('event_dttm')
val=val.sort_values('event_dttm')

In [9]:
524287 in train.customer_id.values

True

In [10]:
train['cust_embs']=train.customer_id.map(lambda x:cust_embs[x][0] )
val['cust_embs']=val.customer_id.map(lambda x:cust_embs[x][0] )

In [11]:
len(train), len(val) 

(320248, 152893)

In [12]:
train=train.fillna(-999)
val=val.fillna(-999)
test=test.fillna(-999)

In [13]:
def reaction_to_weight(reaction):
    if reaction =='dislike':
        return 10.0
#         return 10.0
    if reaction =='like':
        return 0.5
#         return 0.5
    if reaction =='view':
        return 0.1
#         return 0.1
    if reaction =='skip':
        return 0.1
#         return 0.1
    
def reaction_to_class(reaction):
    if reaction =='dislike':
        return -1
    if reaction =='like':
        return 1
    if reaction =='view':
        return 1
    if reaction =='skip':
        return -1
    

In [14]:
train_class = train.event.values
trainY = [reaction_to_class(reaction)  for reaction in train.event.values]
train_weights = [reaction_to_weight(reaction) for reaction in train.event.values]
# del train['event']
val_class = val.event.values
valY = [reaction_to_class(reaction) for reaction in val.event.values]
val_weights = [reaction_to_weight(reaction) for reaction in val.event.values]
# del val['event']



In [15]:
columns = [
    'cust_embs',
                         'gender_cd',
     'job_title',
    'marital_status_cd',
    'job_position_cd',
                         'children_cnt',
                         'story_id',
                         'product_0', 'product_1', 'product_2', 'product_3', 'product_4', 'product_5', 'product_6'
                        ]
cat_columns = [
     'job_title',
    'marital_status_cd',
    'job_position_cd',
               'story_id',
               'product_0', 'product_1', 'product_2', 'product_3', 'product_4', 'product_5', 'product_6',
              ]

val_data = Pool(val[columns],
                valY,
                 cat_features=cat_columns,
               weight=val_weights
               )
train_data = Pool(train[columns],
                  trainY,
                  cat_features=cat_columns,
                 weight=train_weights
                 )

In [20]:
model = CatBoostRegressor(iterations=4000,
                           depth=4, 
#                            scale_pos_weight=1/pds[0].isLiked.mean(),
                           learning_rate=0.05,
                           loss_function='MAE', 
#                           has_time=True,
#                            logging_level='Verbose',
#                            eval_metric='AUC',
#                            bagging_temperature=1.8,
#                            max_bin=128,
#                            random_strength=0,
                           max_ctr_complexity=0,
#                            l2_leaf_reg=10,
# #                            rsm=0.2
#                            use_best_model=False,
                           task_type='GPU',
# #                            used_ram_limit='11GB'
                 
                          )

print('model')
#train the model
model.fit(train_data,
          eval_set=val_data
         )

# 999:	learn: 0.7219157	test: 0.7182027	best: 0.7142853 (747)	total: 5.63s	remaining: 0us
# only customer_id
# train metric 0.3400194482676537
# val metric 0.2889284055655854

# 999:	learn: 0.7134417	test: 0.7149829	best: 0.7147067 (529)	total: 6.47s	remaining: 0us
#  customer_id gender_cd
# train metric 0.3463319497460446
# val metric 0.2875417257135316

# 1999:	learn: 0.7161376	test: 0.7143624	best: 0.7131135 (912)	total: 12.1s	remaining: 0us
#  customer_id gender_cd children_cnt
# train metric 0.3508508692598425
# val metric 0.28944819862286486

model
0:	learn: 0.9930431	test: 0.9955914	best: 0.9955914 (0)	total: 3.86ms	remaining: 15.4s
1:	learn: 0.9860530	test: 0.9917851	best: 0.9917851 (1)	total: 7.56ms	remaining: 15.1s
2:	learn: 0.9790578	test: 0.9879505	best: 0.9879505 (2)	total: 10.8ms	remaining: 14.5s
3:	learn: 0.9720611	test: 0.9836366	best: 0.9836366 (3)	total: 13.8ms	remaining: 13.8s
4:	learn: 0.9651162	test: 0.9796408	best: 0.9796408 (4)	total: 17ms	remaining: 13.6s
5:	learn: 0.9581187	test: 0.9753267	best: 0.9753267 (5)	total: 20.3ms	remaining: 13.5s
6:	learn: 0.9511412	test: 0.9710062	best: 0.9710062 (6)	total: 23.4ms	remaining: 13.3s
7:	learn: 0.9441492	test: 0.9666115	best: 0.9666115 (7)	total: 26.6ms	remaining: 13.3s
8:	learn: 0.9371484	test: 0.9622823	best: 0.9622823 (8)	total: 30ms	remaining: 13.3s
9:	learn: 0.9301786	test: 0.9584026	best: 0.9584026 (9)	total: 33.4ms	remaining: 13.3s
10:	learn: 0.9232125	test: 0.9540178	best: 0.9540178 (10)	total: 36.8ms	remaining: 13.3s
11:	learn: 0.9162562	test: 0.9500207	be

120:	learn: 0.5660615	test: 0.7333735	best: 0.7329292 (115)	total: 398ms	remaining: 12.7s
121:	learn: 0.5654161	test: 0.7344615	best: 0.7329292 (115)	total: 400ms	remaining: 12.7s
122:	learn: 0.5648005	test: 0.7332489	best: 0.7329292 (115)	total: 403ms	remaining: 12.7s
123:	learn: 0.5645035	test: 0.7335273	best: 0.7329292 (115)	total: 405ms	remaining: 12.7s
124:	learn: 0.5638611	test: 0.7329617	best: 0.7329292 (115)	total: 408ms	remaining: 12.6s
125:	learn: 0.5637153	test: 0.7324645	best: 0.7324645 (125)	total: 410ms	remaining: 12.6s
126:	learn: 0.5643399	test: 0.7338732	best: 0.7324645 (125)	total: 414ms	remaining: 12.6s
127:	learn: 0.5635127	test: 0.7336434	best: 0.7324645 (125)	total: 417ms	remaining: 12.6s
128:	learn: 0.5631138	test: 0.7333927	best: 0.7324645 (125)	total: 420ms	remaining: 12.6s
129:	learn: 0.5634681	test: 0.7336565	best: 0.7324645 (125)	total: 424ms	remaining: 12.6s
130:	learn: 0.5626174	test: 0.7321988	best: 0.7321988 (130)	total: 427ms	remaining: 12.6s
131:	learn

232:	learn: 0.5548636	test: 0.7281369	best: 0.7254779 (188)	total: 789ms	remaining: 12.8s
233:	learn: 0.5547656	test: 0.7284233	best: 0.7254779 (188)	total: 792ms	remaining: 12.7s
234:	learn: 0.5547641	test: 0.7281780	best: 0.7254779 (188)	total: 795ms	remaining: 12.7s
235:	learn: 0.5555358	test: 0.7283676	best: 0.7254779 (188)	total: 798ms	remaining: 12.7s
236:	learn: 0.5546520	test: 0.7276801	best: 0.7254779 (188)	total: 804ms	remaining: 12.8s
237:	learn: 0.5545387	test: 0.7279139	best: 0.7254779 (188)	total: 809ms	remaining: 12.8s
238:	learn: 0.5548749	test: 0.7284187	best: 0.7254779 (188)	total: 814ms	remaining: 12.8s
239:	learn: 0.5549588	test: 0.7284189	best: 0.7254779 (188)	total: 819ms	remaining: 12.8s
240:	learn: 0.5540931	test: 0.7273765	best: 0.7254779 (188)	total: 822ms	remaining: 12.8s
241:	learn: 0.5547186	test: 0.7281605	best: 0.7254779 (188)	total: 825ms	remaining: 12.8s
242:	learn: 0.5541368	test: 0.7277344	best: 0.7254779 (188)	total: 830ms	remaining: 12.8s
243:	learn

347:	learn: 0.5513281	test: 0.7283946	best: 0.7248034 (266)	total: 1.18s	remaining: 12.4s
348:	learn: 0.5520871	test: 0.7281204	best: 0.7248034 (266)	total: 1.19s	remaining: 12.4s
349:	learn: 0.5514344	test: 0.7284613	best: 0.7248034 (266)	total: 1.19s	remaining: 12.4s
350:	learn: 0.5517074	test: 0.7292167	best: 0.7248034 (266)	total: 1.19s	remaining: 12.4s
351:	learn: 0.5520638	test: 0.7285754	best: 0.7248034 (266)	total: 1.19s	remaining: 12.4s
352:	learn: 0.5516828	test: 0.7280414	best: 0.7248034 (266)	total: 1.2s	remaining: 12.4s
353:	learn: 0.5514539	test: 0.7273219	best: 0.7248034 (266)	total: 1.21s	remaining: 12.4s
354:	learn: 0.5525042	test: 0.7277910	best: 0.7248034 (266)	total: 1.21s	remaining: 12.4s
355:	learn: 0.5517425	test: 0.7276756	best: 0.7248034 (266)	total: 1.22s	remaining: 12.5s
356:	learn: 0.5517301	test: 0.7275118	best: 0.7248034 (266)	total: 1.22s	remaining: 12.4s
357:	learn: 0.5520192	test: 0.7280076	best: 0.7248034 (266)	total: 1.22s	remaining: 12.4s
358:	learn:

481:	learn: 0.5489554	test: 0.7273464	best: 0.7248034 (266)	total: 1.57s	remaining: 11.5s
482:	learn: 0.5487736	test: 0.7271439	best: 0.7248034 (266)	total: 1.57s	remaining: 11.5s
483:	learn: 0.5488140	test: 0.7273396	best: 0.7248034 (266)	total: 1.58s	remaining: 11.4s
484:	learn: 0.5487760	test: 0.7271600	best: 0.7248034 (266)	total: 1.58s	remaining: 11.4s
485:	learn: 0.5487562	test: 0.7272271	best: 0.7248034 (266)	total: 1.58s	remaining: 11.4s
486:	learn: 0.5487448	test: 0.7272459	best: 0.7248034 (266)	total: 1.58s	remaining: 11.4s
487:	learn: 0.5487254	test: 0.7272272	best: 0.7248034 (266)	total: 1.59s	remaining: 11.4s
488:	learn: 0.5487174	test: 0.7272711	best: 0.7248034 (266)	total: 1.59s	remaining: 11.4s
489:	learn: 0.5487073	test: 0.7272146	best: 0.7248034 (266)	total: 1.59s	remaining: 11.4s
490:	learn: 0.5487174	test: 0.7272906	best: 0.7248034 (266)	total: 1.6s	remaining: 11.4s
491:	learn: 0.5486112	test: 0.7272991	best: 0.7248034 (266)	total: 1.6s	remaining: 11.4s
492:	learn: 

...

3396:	learn: 0.5265350	test: 0.7412216	best: 0.7248034 (266)	total: 10.6s	remaining: 1.88s
3397:	learn: 0.5265386	test: 0.7411016	best: 0.7248034 (266)	total: 10.6s	remaining: 1.88s
3398:	learn: 0.5265331	test: 0.7411329	best: 0.7248034 (266)	total: 10.6s	remaining: 1.87s
3399:	learn: 0.5265301	test: 0.7411179	best: 0.7248034 (266)	total: 10.6s	remaining: 1.87s
3400:	learn: 0.5265275	test: 0.7411007	best: 0.7248034 (266)	total: 10.6s	remaining: 1.87s
3401:	learn: 0.5265188	test: 0.7411202	best: 0.7248034 (266)	total: 10.6s	remaining: 1.86s
3402:	learn: 0.5265149	test: 0.7410597	best: 0.7248034 (266)	total: 10.6s	remaining: 1.86s
3403:	learn: 0.5265126	test: 0.7410714	best: 0.7248034 (266)	total: 10.6s	remaining: 1.86s
3404:	learn: 0.5265157	test: 0.7410889	best: 0.7248034 (266)	total: 10.6s	remaining: 1.85s
3405:	learn: 0.5265121	test: 0.7410603	best: 0.7248034 (266)	total: 10.6s	remaining: 1.85s
3406:	learn: 0.5265120	test: 0.7410671	best: 0.7248034 (266)	total: 10.6s	remaining: 1.85s

3529:	learn: 0.5262109	test: 0.7418714	best: 0.7248034 (266)	total: 11s	remaining: 1.46s
3530:	learn: 0.5262095	test: 0.7418409	best: 0.7248034 (266)	total: 11s	remaining: 1.46s
3531:	learn: 0.5262055	test: 0.7418133	best: 0.7248034 (266)	total: 11s	remaining: 1.46s
3532:	learn: 0.5262029	test: 0.7418097	best: 0.7248034 (266)	total: 11s	remaining: 1.45s
3533:	learn: 0.5261982	test: 0.7417973	best: 0.7248034 (266)	total: 11s	remaining: 1.45s
3534:	learn: 0.5261955	test: 0.7418346	best: 0.7248034 (266)	total: 11s	remaining: 1.45s
3535:	learn: 0.5261938	test: 0.7418447	best: 0.7248034 (266)	total: 11s	remaining: 1.44s
3536:	learn: 0.5261977	test: 0.7416109	best: 0.7248034 (266)	total: 11s	remaining: 1.44s
3537:	learn: 0.5261959	test: 0.7416160	best: 0.7248034 (266)	total: 11s	remaining: 1.44s
3538:	learn: 0.5261912	test: 0.7416154	best: 0.7248034 (266)	total: 11s	remaining: 1.43s
3539:	learn: 0.5261629	test: 0.7416055	best: 0.7248034 (266)	total: 11s	remaining: 1.43s
3540:	learn: 0.526158

3663:	learn: 0.5259897	test: 0.7418585	best: 0.7248034 (266)	total: 11.4s	remaining: 1.04s
3664:	learn: 0.5259879	test: 0.7418767	best: 0.7248034 (266)	total: 11.4s	remaining: 1.04s
3665:	learn: 0.5259744	test: 0.7418547	best: 0.7248034 (266)	total: 11.4s	remaining: 1.04s
3666:	learn: 0.5259684	test: 0.7418107	best: 0.7248034 (266)	total: 11.4s	remaining: 1.03s
3667:	learn: 0.5259722	test: 0.7417741	best: 0.7248034 (266)	total: 11.4s	remaining: 1.03s
3668:	learn: 0.5259663	test: 0.7417798	best: 0.7248034 (266)	total: 11.4s	remaining: 1.03s
3669:	learn: 0.5259624	test: 0.7418066	best: 0.7248034 (266)	total: 11.4s	remaining: 1.02s
3670:	learn: 0.5259572	test: 0.7418495	best: 0.7248034 (266)	total: 11.4s	remaining: 1.02s
3671:	learn: 0.5259577	test: 0.7418421	best: 0.7248034 (266)	total: 11.4s	remaining: 1.02s
3672:	learn: 0.5259599	test: 0.7418614	best: 0.7248034 (266)	total: 11.4s	remaining: 1.01s
3673:	learn: 0.5259588	test: 0.7418951	best: 0.7248034 (266)	total: 11.4s	remaining: 1.01s

3791:	learn: 0.5258070	test: 0.7411962	best: 0.7248034 (266)	total: 11.8s	remaining: 646ms
3792:	learn: 0.5258063	test: 0.7412746	best: 0.7248034 (266)	total: 11.8s	remaining: 642ms
3793:	learn: 0.5258090	test: 0.7412556	best: 0.7248034 (266)	total: 11.8s	remaining: 639ms
3794:	learn: 0.5258062	test: 0.7413215	best: 0.7248034 (266)	total: 11.8s	remaining: 636ms
3795:	learn: 0.5258050	test: 0.7412734	best: 0.7248034 (266)	total: 11.8s	remaining: 633ms
3796:	learn: 0.5258058	test: 0.7412554	best: 0.7248034 (266)	total: 11.8s	remaining: 630ms
3797:	learn: 0.5258025	test: 0.7412580	best: 0.7248034 (266)	total: 11.8s	remaining: 627ms
3798:	learn: 0.5257984	test: 0.7413100	best: 0.7248034 (266)	total: 11.8s	remaining: 624ms
3799:	learn: 0.5257985	test: 0.7412748	best: 0.7248034 (266)	total: 11.8s	remaining: 621ms
3800:	learn: 0.5257965	test: 0.7410164	best: 0.7248034 (266)	total: 11.8s	remaining: 618ms
3801:	learn: 0.5257974	test: 0.7410128	best: 0.7248034 (266)	total: 11.8s	remaining: 615ms

3927:	learn: 0.5256342	test: 0.7422368	best: 0.7248034 (266)	total: 12.2s	remaining: 223ms
3928:	learn: 0.5256304	test: 0.7422440	best: 0.7248034 (266)	total: 12.2s	remaining: 220ms
3929:	learn: 0.5256303	test: 0.7422327	best: 0.7248034 (266)	total: 12.2s	remaining: 217ms
3930:	learn: 0.5256354	test: 0.7422475	best: 0.7248034 (266)	total: 12.2s	remaining: 214ms
3931:	learn: 0.5256358	test: 0.7422813	best: 0.7248034 (266)	total: 12.2s	remaining: 210ms
3932:	learn: 0.5256431	test: 0.7423237	best: 0.7248034 (266)	total: 12.2s	remaining: 207ms
3933:	learn: 0.5256452	test: 0.7422791	best: 0.7248034 (266)	total: 12.2s	remaining: 204ms
3934:	learn: 0.5256481	test: 0.7423050	best: 0.7248034 (266)	total: 12.2s	remaining: 201ms
3935:	learn: 0.5256479	test: 0.7422876	best: 0.7248034 (266)	total: 12.2s	remaining: 198ms
3936:	learn: 0.5256453	test: 0.7423142	best: 0.7248034 (266)	total: 12.2s	remaining: 195ms
3937:	learn: 0.5256445	test: 0.7422682	best: 0.7248034 (266)	total: 12.2s	remaining: 192ms

In [21]:
x = model.get_feature_importance(train_data)
for i in range(len(x)):
    print(columns[i], x[i])

cust_embs 58.29747688709425
gender_cd 7.436333136757987
job_title 0.0
marital_status_cd 0.0
job_position_cd 0.0
children_cnt 0.0
story_id 34.26618997614777
product_0 0.0
product_1 0.0
product_2 0.0
product_3 0.0
product_4 0.0
product_5 0.0
product_6 0.0


In [22]:
train_predict = model.predict(train_data)
val_predict = model.predict(val_data)

print('train metric ' + str(metric(train_predict.clip(-1,1), train_class)))
print('val metric ' + str(metric(val_predict.clip(-1,1), val_class)))

def mae2(predicts, Y, weights):
#     return np.mean(np.multiply(np.abs(Y-predicts), weights))
    max_value = sum(np.abs(np.multiply(Y, weights)))

    sign_weights = np.multiply(Y, weights)
    return np.sum(np.abs(sign_weights-np.multiply(predicts, weights)))/max_value


#     return np.mean(np.multiply(Y,weights)-np.multiply(predicts,weights))
print(mae2(val_predict ,valY, val_weights))
print(mae2(train_predict ,trainY, train_weights))

train metric 0.4525537725512767
val metric 0.27845963286278425
0.7248034350043085
0.5516893640579033


In [22]:
val['predict']=model.predict(val_data)
val.loc[~val.customer_id.isin(set(train.customer_id.values)),'predict'] =-1
print('val metric ' + str(metric(val['predict'].clip(-1,1), val_class)))


val metric 0.28495626177725697


In [100]:
train_weights

[0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.5,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 10.0,
 0.1,
 0.1,
 0.1,
 10.0,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 10.0,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 

In [101]:
def mae2(predicts, Y, weights):
    max_value = sum(np.abs(weights))
    sign_weights = np.multiply(Y, weights)
    return np.sum(np.multiply(sign_weights, -predicts))/max_value
print(mae2(val_predict ,valY, val_weights))

-0.28253761622416307


In [91]:
np.multiply(sign_weights, -predicts)
np.abs(sign_weights-np.multiply(predicts, weights))

NameError: name 'Y' is not defined

In [ ]:
np.multiply(np.multiply(Y,weights), predicts)

0.22792780076833008


0.013162910807036882

In [102]:
mae2(valY, val_weights, val_predict)

0.0747334197951413

In [103]:
mae2(train_predict, trainY, train_weights)

0.2895163131422259

In [65]:
test_id = test.answer_id
del test['answer_id']

In [73]:
test_id

0              0
1              1
2              2
3              3
4              4
           ...  
172044    172044
172045    172045
172046    172046
172047    172047
172048    172048
Name: answer_id, Length: 172049, dtype: int64

In [66]:
test['score'] = (model.predict_proba(test)[:,1]-0.5)*2
test['answer_id'] = test_id

AttributeError: 'CatBoostRegressor' object has no attribute 'predict_proba'

In [119]:
test[['answer_id', 'score']].to_csv("predict.csv", index=False, float_format='%.3f')